# Cycle
This cycle uses mixture experimentalist, BMS theorist, and equation sampler as a source for the ground truth. 

In [3]:
import copy
from dataclasses import dataclass, field
from typing import List

from sklearn.base import BaseEstimator
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import numpy as np
import math
import pandas as pd
from autora.variable import VariableCollection, Variable
from autora.state.bundled import StandardState
from autora.state.delta import on_state
from autora.state.wrapper import state_fn_from_estimator
from autora.theorist.bms import BMSRegressor
from equation_tree import sample 
from equation_tree.tree import instantiate_constants
from equation_tree.prior import DEFAULT_PRIOR_FUNCTIONS, DEFAULT_PRIOR_OPERATORS, \
    structure_prior_from_max_depth
import pprint
from autora.experiment_runner.synthetic.abstract.equation import equation_experiment
from autora.experimentalist.mixture import sample as mixture_sample
from autora.experimentalist.grid_ import grid_pool
from autora.experimentalist.random_ import random_sample
from autora.state.delta import Delta
from autora.experimentalist.falsification import falsification_score_sample
from autora.experimentalist.model_disagreement import model_disagreement_score_sample
from autora.experimentalist.novelty import novelty_score_sample

In [60]:
N_CONDITIONS = 50000
TEMPERATURE = 1.
WEIGHTS = {'falsification':[.1, .1], 'novelty':[.5, .5]}
NUM_SAMPLES = 100

## Ground truth
Sampling the ground truth for this simulation.

In [5]:
structure_prior = structure_prior_from_max_depth(10)
pprint.pprint(structure_prior)
pprint.pprint(DEFAULT_PRIOR_FUNCTIONS)
pprint.pprint(DEFAULT_PRIOR_OPERATORS)
feature_prior = {'constants': .3, 'variables': .7}
prior = {'functions': DEFAULT_PRIOR_FUNCTIONS, 'operators': DEFAULT_PRIOR_OPERATORS, 'structures': structure_prior, 'features': feature_prior}
pprint.pprint(prior)

{'[0, 1, 1, 2, 2, 3, 3, 4, 4, 5]': 0.0007288629737609329,
 '[0, 1, 1, 2, 2, 3, 3, 4, 4]': 0.0007288629737609329,
 '[0, 1, 1, 2, 2, 3, 3, 4, 5, 4]': 0.0007288629737609329,
 '[0, 1, 1, 2, 2, 3, 3, 4, 5, 5]': 0.0007288629737609329,
 '[0, 1, 1, 2, 2, 3, 3, 4, 5, 6]': 0.0007288629737609329,
 '[0, 1, 1, 2, 2, 3, 3, 4, 5]': 0.0007288629737609329,
 '[0, 1, 1, 2, 2, 3, 3, 4]': 0.0007288629737609329,
 '[0, 1, 1, 2, 2, 3, 3]': 0.0007288629737609329,
 '[0, 1, 1, 2, 2, 3, 4, 3, 4, 4]': 0.0007288629737609329,
 '[0, 1, 1, 2, 2, 3, 4, 3, 4, 5]': 0.0007288629737609329,
 '[0, 1, 1, 2, 2, 3, 4, 3, 4]': 0.0007288629737609329,
 '[0, 1, 1, 2, 2, 3, 4, 3]': 0.0007288629737609329,
 '[0, 1, 1, 2, 2, 3, 4, 4, 3, 4]': 0.0007288629737609329,
 '[0, 1, 1, 2, 2, 3, 4, 4, 3]': 0.0007288629737609329,
 '[0, 1, 1, 2, 2, 3, 4, 4, 5, 3]': 0.0007288629737609329,
 '[0, 1, 1, 2, 2, 3, 4, 4, 5, 5]': 0.0007288629737609329,
 '[0, 1, 1, 2, 2, 3, 4, 4, 5, 6]': 0.0007288629737609329,
 '[0, 1, 1, 2, 2, 3, 4, 4, 5]': 0.0007288629737

In [19]:
equation_raw = sample(n=1, prior=prior, max_num_variables=4)
equation_raw[0].sympy_expr

Processing: 100%|██████████| 1/1 [00:00<00:00, 15.56iteration/s]


-cos(cos(sin(x_1))) + Abs(x_1 - Abs(x_2))

In [20]:
equation = instantiate_constants(equation_raw[0], lambda: np.random.rand()*100)
equation.sympy_expr


-cos(cos(sin(x_1))) + Abs(x_1 - Abs(x_2))

In [201]:
features = {'x_1': np.linspace(-10, 10, 100), 'x_2': np.linspace(1, 11, 100), 'x_3': np.linspace(1, 11, 100)}

In [67]:
equation_raw[0]._evaluate(features)
equation_raw[0].has_valid_value

KeyError: 'c_1'

Defining the metadata based on the sampled ground truth.

In [21]:
independent_variables = [
    Variable("x_1", allowed_values=np.linspace(-10, 10, 15)),
    Variable("x_2", allowed_values=np.linspace(1, 11, 50000)),
    #Variable("x_3", allowed_values=np.linspace(1, 11, 15)),
]
# for v in range(equation.n_variables_unique):
#     # taking a floor depending on n of variables so that each experimental space has roughly the same coarseness
#     independent_variables.append(Variable(f"x_{v+1}",allowed_values=np.linspace(-10, 10, math.floor(N_CONDITIONS**(1/equation.n_variables_unique)))))



variables=VariableCollection(
        independent_variables=independent_variables,
        dependent_variables=[Variable("y")]
    )

Defining experiment runner from the equation and the variable collection

In [22]:
experiment = equation_experiment(equation.sympy_expr, variables.independent_variables, variables.dependent_variables[0], rename_output_columns=False)

### Defining the state
We can define an initial state for our discovery problem based on the variable specification above. Wrapping experiment runner into the state.

In [50]:
@dataclass(frozen=True)
class ExtendedState(StandardState):
    models_bms: List[BaseEstimator] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    )
    models_linear: List[BaseEstimator] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    )
    models_polynom: List[BaseEstimator] = field(
        default_factory=list,
        metadata={"delta": "extend"},
    )

state = ExtendedState(
    variables=variables
)
runner_on_state = on_state(experiment.experiment_runner, output=["experiment_data"])

### Pooler

In [51]:
experimentalist_pooler = on_state(grid_pool, output=["conditions"])

## Mixture experimentalist
Defining the mixture experimentalist and wrapping it into the state

In [97]:
"""
Mixture Experimentalist Sampler
"""

import numpy as np
from typing import Optional, Union

import pandas as pd


def adjust_distribution(p_, temperature):
    # temperature cannot be 0
    assert temperature != 0, 'Temperature cannot be 0'
    p = np.array(p_)

    print(p)
    # If the temperature is very low (close to 0), then the sampling will become almost deterministic, picking the event with the highest probability.
    # If the temperature is very high, then the sampling will be closer to uniform, with all events having roughly equal probability.

    p = p / np.sum(np.abs(p))  # Normalizing the initial distribution
    print(p)
    print(temperature)
    p = np.exp(p / temperature)
    print('***')
    print(p)
    final_p = p / np.sum(p)  # Normalizing the final distribution
    return final_p


def sample(conditions: Union[pd.DataFrame, np.ndarray], temperature: float,
                   samplers: list, params: dict,
                   num_samples: Optional[int] = None) -> pd.DataFrame:
    """

    Args:
        conditions: pool of experimental conditions to evaluate: pd.Dataframe
        temperature: how random is selection of conditions (cannot be 0; (0:1) - the choices are more deterministic than the choices made wrt
        samplers: tuple containing sampler functions, their names, and weights
        for sampler functions that return both positive and negative scores, user can provide a list with two weights: the first one will be applied to positive scores, the second one -- to the negative
        params: nested dictionary. keys correspond to the sampler function names (same as provided in samplers),
        values correspond to the dictionaries of function arguments (argument name: its value)
        num_samples: number of experimental conditions to select

    Returns:
        Sampled pool of experimental conditions with the scores attached to them
    """

    condition_pool = pd.DataFrame(conditions)

    rankings = pd.DataFrame()
    mixture_scores = np.zeros(len(condition_pool))
    ## getting rankings and weighted scores from each function
    for (function, name, weight) in samplers:
        try:
            sampler_params = params[name]
            pd_ranking = function(conditions=condition_pool, **sampler_params)
        except:
            pd_ranking = function(conditions=condition_pool)
        # sorting by index
        pd_ranking = pd_ranking.sort_index()
        # if only one weight is provided, use it for both negative and positive dimensions
        if isinstance(weight, float) or isinstance(weight, int):
            pd_ranking["score"] = pd_ranking["score"] * weight
        else:
            if len(pd_ranking["score"] < 0) > 0 and len(pd_ranking["score"] > 0) > 0:  # there are both positive and negative values

                pd_ranking.loc[pd_ranking["score"] > 0]["score"] = pd_ranking.loc[pd_ranking["score"] > 0]["score"] * weight[0]  # positive dimension gets the first weight
                pd_ranking.loc[pd_ranking["score"] < 0]["score"] = pd_ranking.loc[pd_ranking["score"] < 0]["score"] * weight[1]  # negative dimension gets the second weight
            else:
                pd_ranking["score"] = pd_ranking["score"] * weight[0]

        pd_ranking.rename(columns={"score": f"{name}_score"}, inplace=True)
        # sum_scores are arranged based on the original conditions_ indices
        mixture_scores = mixture_scores + pd_ranking[f"{name}_score"]

        rankings = pd.merge(rankings, pd_ranking, left_index=True, right_index=True, how="outer")

    # adjust mixture scores wrt temperature
    weighted_mixture_scores_adjusted = adjust_distribution(mixture_scores, temperature)
    print(weighted_mixture_scores_adjusted)

    if num_samples is None:
        num_samples = condition_pool.shape[0]

    condition_indices = np.random.choice(np.arange(len(condition_pool)), num_samples,
                                         p=weighted_mixture_scores_adjusted, replace=False)
    conditions_ = condition_pool.iloc[condition_indices]
    conditions_["score"] = mixture_scores

    return conditions_


mixture_sample_test = sample

In [92]:
@on_state()
def experimentalist_sample(conditions, models, experiment_data, variables, temperature, weights, num_samples):
    print(models)
    print(experiment_data)
    if models is None or experiment_data is None:
        print('First cycle: Using random sampler')
        conditions_ = random_sample(conditions, num_samples)
    else:
        experiment_conditions = experiment_data[[v.name for v in variables.independent_variables]]
        experiment_observations = experiment_data[[v.name for v in variables.dependent_variables]]
        params_ = {} #copy.deepcopy(params)
        params_["falsification"] = {"reference_conditions": experiment_conditions, "reference_observations": experiment_observations, "model": models[-1]}
        params_["novelty"] = {"reference_conditions": experiment_conditions}


        samplers = [
            [novelty_score_sample, "novelty", weights["novelty"]],
            #[falsification_score_sample, "falsification", weights["falsification"]]
        ]
        print(samplers)


        conditions_ = mixture_sample_test(conditions, temperature, samplers, params_, num_samples)
        conditions_ = conditions_.drop("score", axis = 1)
    #d = Delta(conditions=conditions)
    d = Delta(conditions = conditions_)
    return d

## BMS theorist
Defining the BMS theorist and wrapping it into the state

In [76]:
@on_state()
def bms_theorist(experiment_data: pd.DataFrame, variables: VariableCollection, **kwargs):
    ivs = [v.name for v in variables.independent_variables]
    dvs = [v.name for v in variables.dependent_variables]
    X, y = experiment_data[ivs], experiment_data[dvs]
    new_model = BMSRegressor(epochs=10).set_params(**kwargs).fit(X, y)
    return Delta(models_bms=[new_model])

@on_state()
def linear_theorist(experiment_data: pd.DataFrame, variables: VariableCollection, **kwargs):
    ivs = [v.name for v in variables.independent_variables]
    dvs = [v.name for v in variables.dependent_variables]
    X, y = experiment_data[ivs], experiment_data[dvs]
    new_model = LinearRegression().set_params(**kwargs).fit(X, y)
    return Delta(models_linear=[new_model])


def PolynomialRegression(degree=3, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), LinearRegression(**kwargs))


@on_state()
def polynomial_theorist(experiment_data: pd.DataFrame, variables: VariableCollection, **kwargs):
    ivs = [v.name for v in variables.independent_variables]
    dvs = [v.name for v in variables.dependent_variables]
    X, y = experiment_data[ivs], experiment_data[dvs]
    new_model = PolynomialRegression()
    new_model.fit(X, y)
    return Delta(models_polynom=[new_model])


In [54]:
@on_state()
def best_model(models_bms, models_linear, models_polynom, experiment_data, variables):
    ivs = [v.name for v in variables.independent_variables]
    dvs = [v.name for v in variables.dependent_variables]
    X, y = experiment_data[ivs], experiment_data[dvs]
    prediction_bms = models_bms[-1].predict(X)
    prediction_linear = models_linear[-1].predict(X)
    prediction_polynomial = models_polynom[-1].predict(X)
    mad_bms = mean_absolute_error(y, prediction_bms)
    mad_linear = mean_absolute_error(y, prediction_linear)
    mad_poly = mean_absolute_error(y, prediction_polynomial)
    if mad_bms <= mad_linear and mad_bms <= mad_poly:
        new_model = models_bms[-1]
    elif mad_linear <= mad_bms and mad_linear <= mad_poly:
        new_model = models_linear[-1]
    elif mad_poly <= mad_linear and mad_poly <= mad_bms:
        new_model = models_polynom[-1]

    return Delta(model=new_model)


In [95]:
def cycle(s):
    s_pool = experimentalist_pooler(s)
    s_conditions = experimentalist_sample(s_pool, temperature=TEMPERATURE, weights=WEIGHTS, num_samples=NUM_SAMPLES)
    s_run = runner_on_state(s_conditions)
    s_theory = bms_theorist(s_run)
    s_theory = linear_theorist(s_theory)
    s_theory = polynomial_theorist(s_theory)
    s_best = best_model(s_theory)
    return s_best


In [98]:
state = ExtendedState(variables=variables)
for _ in range(10):
    state = cycle(state)

print(state)


INFO:autora.theorist.bms.regressor:BMS fitting started


[]
None
First cycle: Using random sampler


100%|██████████| 10/10 [00:00<00:00, 31.51it/s]
INFO:autora.theorist.bms.regressor:BMS fitting finished


[Pipeline(steps=[('polynomialfeatures', PolynomialFeatures(degree=3)),
                ('linearregression', LinearRegression())])]
              x_1        x_2          y
302871  -1.428571   1.574211   2.153851
509748   4.285714   2.949639   0.523436
449428   1.428571  10.885798   8.589889
390257   0.000000   9.051561   8.515197
468775   2.857143   4.755075   1.315494
...           ...        ...        ...
292084  -2.857143   9.416968  11.709568
3672   -10.000000   1.734415  11.069218
441144   1.428571   9.228965   6.944449
395992   0.000000  10.198584   9.656422
138214  -7.142857   8.642953  15.023901

[100 rows x 3 columns]
[[<function score_sample at 0x294af22a0>, 'novelty', [0.5, 0.5]]]
[2.48475596 2.48469481 2.48463366 ... 2.07699098 2.07705337 2.07711576]
[3.89109944e-06 3.89100367e-06 3.89090791e-06 ... 3.25254414e-06
 3.25264184e-06 3.25273954e-06]
1.0
***
[1.00000389 1.00000389 1.00000389 ... 1.00000325 1.00000325 1.00000325]
[1.33333852e-06 1.33333852e-06 1.33333852e-06 ... 

INFO:autora.theorist.bms.regressor:BMS fitting started
100%|██████████| 10/10 [00:00<00:00, 32.96it/s]
INFO:autora.theorist.bms.regressor:BMS fitting finished


...

INFO:autora.theorist.bms.regressor:BMS fitting started


[2.19688734 2.19682406 2.19676079 ... 2.36345799 2.36352061 2.36358323]
[3.44122267e-06 3.44112356e-06 3.44102445e-06 ... 3.70214033e-06
 3.70223842e-06 3.70233652e-06]
1.0
***
[1.00000344 1.00000344 1.00000344 ... 1.0000037  1.0000037  1.0000037 ]
[1.33333792e-06 1.33333792e-06 1.33333792e-06 ... 1.33333827e-06
 1.33333827e-06 1.33333827e-06]


  0%|          | 0/10 [00:00<?, ?it/s]

Unexpected exception formatting exception. Falling back to standard exception



Traceback (most recent call last):
  File "/Users/younesstrittmatter/Documents/GitHub/AutoResearch/autora-core/venv/lib/python3.11/site-packages/autora/theorist/bms/mcmc.py", line 797, in update_representative
    rep, rep_energy, rep_par_values = self.representative[canonical]
                                      ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^
KeyError: 'x_1+x_2'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/younesstrittmatter/Documents/GitHub/AutoResearch/autora-core/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/1b/73431t791q1_7tb99v1jc9ww0000gp/T/ipykernel_94532/3773975987.py", line 3, in <module>
    state = cycle(state)
            ^^^^^^^^^^^^
  File "/var/folders/1b/73431t791q1_7tb99v1jc9ww0000gp/T/ipykernel_94532/2506226983.py", line 5, in cycle
    s_theory = bms_theorist(s_run)
    